In [5]:
from pathlib import Path
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
from transformer_sim import Config, TSTransformer
#import tqdm
import argparse
import metrics
import os   

fig_path = Path("fig")
fig_path.mkdir(exist_ok=True)

torch.manual_seed(420)
np.random.seed(430)
torch.manual_seed(420)
np.random.seed(430)

# Overall settings
out_dir = "out"
# System settings
nu = 7
ny = 14
batch_size = 32 # 256

# Compute settings
cuda_device = "cuda:0"
no_cuda = False
threads = 5
compile = False 

# Configure compute
torch.set_num_threads(threads) 
use_cuda = not no_cuda and torch.cuda.is_available()
device_name  = cuda_device if use_cuda else "cpu"
device = torch.device(device_name)
device_type = 'cuda' if 'cuda' in device_name else 'cpu' # for later use in torch.autocast
# torch.set_float32_matmul_precision("high") 
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn


In [6]:
parent_folder = os.path.join(os.getcwd(), os.pardir) 
parent_folder = os.path.abspath(parent_folder)

try:
    relative_folder = f"isaacgym/python/examples/Franka/out_tensors/train"
    tensors_path = os.path.join(parent_folder,relative_folder ) 
    tensors_path = os.path.abspath(tensors_path)
    list_of_available_tensors = os.listdir(tensors_path)
except FileNotFoundError:
    relative_folder = f"Data_generation/python/examples/Franka/out_tensors/train"
    tensors_path = os.path.join(parent_folder,relative_folder ) 
    tensors_path = os.path.abspath(tensors_path)
    list_of_available_tensors = os.listdir(tensors_path)

tensors_path = os.path.join(parent_folder,relative_folder ) 
tensors_path = os.path.abspath(tensors_path)
directory_available_tensors = os.listdir(tensors_path)

print("There are "+str(len(directory_available_tensors)) +" available files in "+str(tensors_path) +":\n")

list_available_tensors =[]
for i in range(len(directory_available_tensors)):
    single_pt_file_path = os.path.join(tensors_path, directory_available_tensors[i]) 
    list_available_tensors.append(directory_available_tensors[i])
    print(" --> ",directory_available_tensors[i])

There are 9 available files in /home/manuel/isaacgym/python/examples/Franka/out_tensors/train:

 -->  672_envs_4022_steps_1000_f_0_2_MS_rand_1010_bounds_mass_10_10.pt
 -->  881_envs_4029_steps_1000_f_0_15_MS_rand_1010_bounds_mass_15_15.pt
 -->  205_envs_4035_steps_1000_f_0_15_MS_rand_1010_bounds_mass_10_10.pt
 -->  475_envs_4022_steps_1000_f_0_2_MS_rand_1010_bounds_mass_15_15.pt
 -->  113_envs_4016_steps_1000_f_0_2_MS_rand_1010_bounds_mass_5_5.pt
 -->  11_envs_4037_steps_1000_f_0_1_MS_rand_1010_bounds_mass_5_5.pt
 -->  842_envs_4053_steps_1000_f_0_1_MS_rand_1010_bounds_mass_15_15.pt
 -->  579_envs_4013_steps_1000_f_0_15_MS_rand_1010_bounds_mass_5_5.pt
 -->  534_envs_4037_steps_1000_f_0_1_MS_rand_1010_bounds_mass_10_10.pt


In [7]:
number_of_pt_files = len(directory_available_tensors)
tensors_used_for_training = list_available_tensors[0:number_of_pt_files-1]

In [8]:
pick_number = 3
for i in range(0,number_of_pt_files,pick_number):
    # print("--------------BIG PICK---------------")
    control_action = torch.zeros(0,1000,7)
    position = torch.zeros(0,1000,14)

    for j in range(pick_number):
        single_pt_file_path = os.path.join(tensors_path, directory_available_tensors[i+j]) 
        # print("\nStarting with file:\n", directory_available_tensors[i+j],"\n")
        loaded = torch.load(single_pt_file_path,map_location=device)

        def loading():
            control_action_extracted = loaded['control_action'][1:,:,:7]
            position_extracted = loaded['position']
            position = torch.movedim(position_extracted.to('cpu'),-2,-3)
            control_action = torch.movedim(control_action_extracted.to('cpu'),-2,-3)
            
            control_action = control_action[:,:,:7] # first is the number of simulations
            position = position[:,:,:]
            return control_action,position
        
        control_action_single,position_single = loading()

        control_action = torch.cat((control_action,control_action_single),dim=0)
        position = torch.cat((position,position_single),dim=0)
        
        print("single-control", control_action_single.shape)
        print("single-position", position_single.shape)
        print("\n")
        print("total-control", control_action.shape)
        print("total-position", position.shape)
        print(" ------------------------- \n")

        dim = 0
        idx = torch.randperm(control_action.shape[dim])
        # print(idx)
        control_action_shuffled = control_action[idx,:,:]
        position = position[idx,:,:]
    
        # print("total-control-shuffled", control_action_shuffled.shape)

        # print(control_action[:2,10,:])
        # print(control_action_shuffled[:2,10,:])

    print("i'm training:",)


single-control torch.Size([4022, 1000, 7])
single-position torch.Size([4022, 1000, 14])


total-control torch.Size([4022, 1000, 7])
total-position torch.Size([4022, 1000, 14])
 ------------------------- 

single-control torch.Size([4029, 1000, 7])
single-position torch.Size([4029, 1000, 14])


total-control torch.Size([8051, 1000, 7])
total-position torch.Size([8051, 1000, 14])
 ------------------------- 

single-control torch.Size([4035, 1000, 7])
single-position torch.Size([4035, 1000, 14])


total-control torch.Size([12086, 1000, 7])
total-position torch.Size([12086, 1000, 14])
 ------------------------- 

single-control torch.Size([4022, 1000, 7])
single-position torch.Size([4022, 1000, 14])


total-control torch.Size([4022, 1000, 7])
total-position torch.Size([4022, 1000, 14])
 ------------------------- 

single-control torch.Size([4016, 1000, 7])
single-position torch.Size([4016, 1000, 14])


total-control torch.Size([8038, 1000, 7])
total-position torch.Size([8038, 1000, 14])
 -

In [9]:
# a = torch.rand(2,10,7)
# print(a)
# dim = 0
# idx = torch.randperm(a.shape[dim])
# print("\n")
# a = a[idx,:,:]
# print(a)

In [23]:
pick = 3
for i in range(0,9,2):
    # print(i)
    control = []
    for j in range(pick):
        # print(i+j)
        control.append(i+j)
        
    print("i'm training",control)

i'm training [0, 1, 2]
i'm training [2, 3, 4]
i'm training [4, 5, 6]
i'm training [6, 7, 8]
i'm training [8, 9, 10]
